# A multivariate regression problem

In [0]:
import pandas
import numpy as np
import sklearn.linear_model as lm
from sklearn.model_selection import KFold
from sklearn import preprocessing as pre
import random
import matplotlib.pyplot as plt

Load in the California housing dataset. Originally downloaded from https://github.com/ageron/handson-ml/tree/master/datasets/housing

In [0]:
housing = pandas.read_csv('./housing.csv')

FileNotFoundError: ignored

In [0]:
display(housing)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


This dataset has some special features which need some attention:
1. The variable ocean_proximity is a categorical variable
2. There are some missing values in the total_bedrooms column
Let us consider how to deal with each of these.

First, deal with the categorical variable ocean_proximity. How many distinct values does this take?

In [0]:
print(set(housing.ocean_proximity.values))
print(len(set(housing.ocean_proximity.values)))

{'ISLAND', 'NEAR OCEAN', 'INLAND', '<1H OCEAN', 'NEAR BAY'}
5


There are five unique values and so we will replace this with a one-hot vector of length 5 where 10000 corresponds to <1H OCEAN, 01000 is ISLAND etc. This is a standard way to represent categorical factors. The key is that for any entry (row) only one element of this vector should be 1 - hence one-hot.

In [0]:
housing['1h_ocean'] = [1 if i=='<1H OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['island'] = [1 if i=='ISLAND' else 0 for i in housing.ocean_proximity.values]
housing['inland'] = [1 if i=='INLAND' else 0 for i in housing.ocean_proximity.values]
housing['near_ocean'] = [1 if i=='NEAR OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['near_bay'] = [1 if i=='NEAR BAY' else 0 for i in housing.ocean_proximity.values]
housing.drop(columns=['ocean_proximity'], inplace=True)
display(housing)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,1h_ocean,island,inland,near_ocean,near_bay
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,0,0,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,0,0,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,0,0,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,0,0,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,0,0,1
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,0,0,0,0,1
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,0,0,0,0,1
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,0,0,0,0,1
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,0,0,0,0,1
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,0,0,0,0,1


Now we need to look at the missing values in total_bedrooms, of which there are:

In [0]:
sum(housing.total_bedrooms.isna())

207

How can we deal with this? It depends. There are several strategies one can use.

* Replace with the average of the column, but this loses information about correlation
* Replace with the values from the nearest neighbour, based on the values of the other variables.
* Use some prior knowledge.

We will use some prior knowledge that it is highly likely that the number of bedrooms is strongly correlated with the total number of rooms. We'll fit a linear model to predict the missing values.

In [0]:
# Get the non-Nan indices
notna = housing.total_bedrooms.notna()

In [0]:
model = lm.LinearRegression()
model.fit(housing.total_rooms.values[notna].reshape(-1,1), housing.total_bedrooms.values[notna].reshape(-1,1))
model.score(housing.total_rooms.values[notna].reshape(-1,1), housing.total_bedrooms.values[notna].reshape(-1,1))

0.8656060227407114

This is a strong prediction so our intuition is correct. Now we predict the missing values.

In [0]:
isna = housing.total_bedrooms.isna()
missing_bedrooms = model.predict(housing.total_rooms.values[isna].reshape(-1,1))

Insert the imputed values into the table.

In [0]:
# Can ignore subsequent warning
housing.total_bedrooms.loc[isna] = np.squeeze(missing_bedrooms)

C:\Users\ibs\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
display(housing)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,1h_ocean,island,inland,near_ocean,near_bay
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,0,0,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,0,0,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,0,0,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,0,0,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,0,0,1
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,0,0,0,0,1
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,0,0,0,0,1
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,0,0,0,0,1
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,0,0,0,0,1
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,0,0,0,0,1


So we now have a complete dataset. Can we model it? An obvious thing to try first is a linear model, in which the house price is a weighted sum of the inputs; that is:
$$y = w_0 + w_1 x_1 + w_2 x_2 + w_M x_M$$
Let's assess this via cross-validation

In [0]:
# First, extract the data into arrays
y = housing.median_house_value.values.reshape(-1,1)
X = housing.drop(columns=['median_house_value'], inplace=False).values
print(X.shape)
print(y.shape)
# Pull out 1000 values into a holdout set
holdout = random.sample(range(0,10640),1000)
X_holdout = X[holdout]
y_holdout = y[holdout]
Xt = np.delete(X, holdout, 0)
yt = np.delete(y, holdout, 0)
print(Xt.shape)
print(yt.shape)

(20640, 13)
(20640, 1)
(19640, 13)
(19640, 1)


In [0]:
Model = lm.LinearRegression()
# Have to shuffle the data because it is grouped.
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(Xt):
    X_train, X_test = Xt[train_index], Xt[test_index]
    y_train, y_test = yt[train_index], yt[test_index]
    Model.fit(X_train, y_train)
    print('Training error: ' + str(Model.score(X_train, y_train)))
    print('Testing error: ' + str(Model.score(X_test, y_test)))

Training error: 0.6524072513083177
Testing error: 0.6230704943533701
Training error: 0.6438046521813467
Testing error: 0.6568344893569786
Training error: 0.6472953995412476
Testing error: 0.6439422761000693
Training error: 0.6480745003952236
Testing error: 0.6404296308878297
Training error: 0.6423326165207981
Testing error: 0.6619761142306679


Scores quoted are $R^2$ (coefficient of determination) values which range from 0 to 1. These are OK but there is much room for improvement and we ought to be able to do much better than this. Options that we could try are:

* Normalising/rescaling the data so that all variables have similar values?
* Expanding the basis to include terms that are non-linear in the variables?
* Removing redundant variables from the data - are there any that are correlated with each other?
* Regularisation?

This will be the task of the group assignment.